# ⚖️ L2 Regularization (Weight Decay)

This notebook demonstrates **L2 Regularization** (also known as Weight Decay)—a technique that penalizes large weights to prevent overfitting. We'll compare models trained with and without L2 regularization on the Sonar dataset.

## What You'll Learn

1. How L2 regularization works mathematically
2. Using `weight_decay` in PyTorch optimizers
3. The effect of L2 on training dynamics
4. Choosing the right regularization strength

---

## 1. Setup and Imports

In [ ]:
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

## 2. Understanding L2 Regularization

### The Math Behind L2

L2 regularization adds a penalty term to the loss function:

```
Total Loss = Original Loss + λ × Σ(w²)

Where:
- Original Loss: CrossEntropy, MSE, etc.
- λ (lambda): Regularization strength (weight_decay in PyTorch)
- Σ(w²): Sum of squared weights
```

### Why It Works

By penalizing large weights:
- Model can't rely too heavily on any single feature
- Weights are distributed more evenly
- Simpler, more generalizable patterns are learned

### Effect on Weights

```
Without L2: weights = [2.5, -3.1, 0.8, 4.2, -1.9]  (some very large)
With L2:    weights = [1.2, -1.5, 0.4, 2.0, -0.9]  (smaller, more uniform)
```

## 3. Loading the Sonar Dataset

In [ ]:
df = pd.read_csv("sonar.all-data", header=None)
print(f"Dataset shape: {df.shape}")
print(f"\nClass distribution:")
print(df[60].value_counts())

## 4. Data Preprocessing

In [ ]:
# Convert labels to numeric
df[60] = df[60].map({'M': 0, 'R': 1})

# Split features and target
X = df.drop(60, axis=1)
y = df[60]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=1
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

In [ ]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

---

## 5. Model Definition

In [ ]:
class NeuralNet(nn.Module):
    """Simple neural network for binary classification."""
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(60, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        return self.network(x)

## 6. Training Function

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=40):
    """
    Train model and track metrics.
    
    Returns:
        train_losses, val_losses, val_accuracies
    """
    train_losses, val_losses, val_accuracies = [], [], []
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(train_loader))
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        y_pred, y_true = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                y_pred.extend(predicted.cpu().numpy())
                y_true.extend(labels.cpu().numpy())
        
        val_losses.append(val_loss / len(val_loader))
        val_accuracy = accuracy_score(y_true, y_pred)
        val_accuracies.append(val_accuracy)
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1:2d}/{epochs} | "
                  f"Train Loss: {train_losses[-1]:.4f} | "
                  f"Val Loss: {val_losses[-1]:.4f} | "
                  f"Val Acc: {val_accuracy:.4f}")
    
    return train_losses, val_losses, val_accuracies

---

## 7. Training Without Regularization

First, let's train a model **without any regularization** to establish a baseline and observe overfitting.

In [ ]:
# Initialize model WITHOUT regularization
model_without_reg = NeuralNet()
criterion = nn.CrossEntropyLoss()

# Note: weight_decay=0 means NO L2 regularization
optimizer = optim.Adam(model_without_reg.parameters(), lr=0.001, weight_decay=0)

print("Training WITHOUT L2 Regularization (weight_decay=0)")
print("=" * 60)
train_losses_no_reg, val_losses_no_reg, val_accuracies_no_reg = train_model(
    model_without_reg, train_loader, test_loader, criterion, optimizer, epochs=40
)

In [ ]:
# Visualize training without regularization
plt.figure(figsize=(10, 6))
epochs = range(1, 41)

plt.plot(epochs, train_losses_no_reg, 'b-', label="Train Loss", linewidth=2)
plt.plot(epochs, val_losses_no_reg, 'r-', label="Validation Loss", linewidth=2)
plt.plot(epochs, val_accuracies_no_reg, 'g--', label="Validation Accuracy", linewidth=2)

plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss / Accuracy', fontsize=12)
plt.title('Training WITHOUT L2 Regularization', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

### Analyzing the Results

Look for signs of **overfitting**:
- Training loss keeps decreasing toward zero
- Validation loss may increase or plateau
- Growing gap between training and validation loss

---

## 8. Training With L2 Regularization

Now let's add L2 regularization using the `weight_decay` parameter in the optimizer.

### Using weight_decay in PyTorch

```python
# L2 regularization is built into the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.04)
```

The `weight_decay` parameter adds L2 penalty automatically during optimization.

### Choosing weight_decay Value

| Value | Effect |
|-------|--------|
| 0 | No regularization |
| 1e-5 to 1e-4 | Light regularization |
| 1e-4 to 1e-3 | Moderate regularization |
| 1e-3 to 1e-2 | Strong regularization |
| > 0.01 | Very strong (may underfit) |

In [ ]:
# Initialize model WITH L2 regularization
model_with_l2 = NeuralNet()

# weight_decay=0.04 adds L2 regularization
optimizer_l2 = optim.Adam(model_with_l2.parameters(), lr=0.001, weight_decay=0.04)

print("\nTraining WITH L2 Regularization (weight_decay=0.04)")
print("=" * 60)
train_losses_with_l2, val_losses_with_l2, val_accuracies_with_l2 = train_model(
    model_with_l2, train_loader, test_loader, criterion, optimizer_l2, epochs=40
)

In [ ]:
# Visualize training with L2 regularization
plt.figure(figsize=(10, 6))
epochs = range(1, 41)

plt.plot(epochs, train_losses_with_l2, 'b-', label="Train Loss", linewidth=2)
plt.plot(epochs, val_losses_with_l2, 'r-', label="Validation Loss", linewidth=2)
plt.plot(epochs, val_accuracies_with_l2, 'g--', label="Validation Accuracy", linewidth=2)

plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss / Accuracy', fontsize=12)
plt.title('Training WITH L2 Regularization (weight_decay=0.04)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

---

## 9. Comparing Both Models

Let's compare the training dynamics and final performance of both models.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
epochs = range(1, 41)

# Training Loss Comparison
axes[0].plot(epochs, train_losses_no_reg, 'b-', label="No Regularization", linewidth=2)
axes[0].plot(epochs, train_losses_with_l2, 'r-', label="With L2 (λ=0.04)", linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Training Loss', fontsize=12)
axes[0].set_title('Training Loss Comparison', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Validation Loss Comparison
axes[1].plot(epochs, val_losses_no_reg, 'b-', label="No Regularization", linewidth=2)
axes[1].plot(epochs, val_losses_with_l2, 'r-', label="With L2 (λ=0.04)", linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Validation Loss', fontsize=12)
axes[1].set_title('Validation Loss Comparison', fontsize=14)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Validation Accuracy Comparison
plt.figure(figsize=(10, 6))

plt.plot(epochs, val_accuracies_no_reg, 'b-o', label="No Regularization", 
         linewidth=2, markersize=4)
plt.plot(epochs, val_accuracies_with_l2, 'r-o', label="With L2 (λ=0.04)", 
         linewidth=2, markersize=4)

plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Validation Accuracy', fontsize=12)
plt.title('Validation Accuracy: L2 Regularization Effect', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

## 10. Examining Weight Magnitudes

Let's compare the weight magnitudes of both models to see L2's effect.

In [ ]:
def get_weight_stats(model):
    """Get statistics about model weights."""
    all_weights = []
    for name, param in model.named_parameters():
        if 'weight' in name:
            all_weights.extend(param.detach().cpu().numpy().flatten())
    
    weights = np.array(all_weights)
    return {
        'mean': np.mean(np.abs(weights)),
        'max': np.max(np.abs(weights)),
        'std': np.std(weights),
        'weights': weights
    }

stats_no_reg = get_weight_stats(model_without_reg)
stats_with_l2 = get_weight_stats(model_with_l2)

print("Weight Statistics")
print("=" * 50)
print(f"\nWithout L2 Regularization:")
print(f"  Mean |weight|: {stats_no_reg['mean']:.4f}")
print(f"  Max |weight|:  {stats_no_reg['max']:.4f}")
print(f"  Std:           {stats_no_reg['std']:.4f}")

print(f"\nWith L2 Regularization:")
print(f"  Mean |weight|: {stats_with_l2['mean']:.4f}")
print(f"  Max |weight|:  {stats_with_l2['max']:.4f}")
print(f"  Std:           {stats_with_l2['std']:.4f}")

In [ ]:
# Visualize weight distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(stats_no_reg['weights'], bins=50, alpha=0.7, color='blue')
axes[0].set_xlabel('Weight Value', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Weight Distribution (No Regularization)', fontsize=14)
axes[0].axvline(x=0, color='red', linestyle='--')

axes[1].hist(stats_with_l2['weights'], bins=50, alpha=0.7, color='green')
axes[1].set_xlabel('Weight Value', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_title('Weight Distribution (With L2)', fontsize=14)
axes[1].axvline(x=0, color='red', linestyle='--')

plt.tight_layout()
plt.show()

print("\n💡 Notice how L2 regularization produces smaller, more concentrated weights!")

---

## 11. Summary

In [ ]:
print("\n" + "=" * 50)
print("SUMMARY")
print("=" * 50)

print(f"\nWithout L2 Regularization:")
print(f"  Best Val Accuracy: {max(val_accuracies_no_reg):.4f}")
print(f"  Final Val Accuracy: {val_accuracies_no_reg[-1]:.4f}")
print(f"  Final Train Loss: {train_losses_no_reg[-1]:.4f}")
print(f"  Final Val Loss: {val_losses_no_reg[-1]:.4f}")

print(f"\nWith L2 Regularization (λ=0.04):")
print(f"  Best Val Accuracy: {max(val_accuracies_with_l2):.4f}")
print(f"  Final Val Accuracy: {val_accuracies_with_l2[-1]:.4f}")
print(f"  Final Train Loss: {train_losses_with_l2[-1]:.4f}")
print(f"  Final Val Loss: {val_losses_with_l2[-1]:.4f}")

---

## 12. Key Takeaways

### What L2 Regularization Does

| Effect | Description |
|--------|-------------|
| **Shrinks weights** | Pushes weights toward zero |
| **Prevents overfitting** | Model can't memorize training data |
| **Smoother decision boundaries** | Less complex model |
| **Better generalization** | Works better on unseen data |

### PyTorch Implementation

```python
# Method 1: Built into optimizer (recommended)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Method 2: Use AdamW for proper weight decay with Adam
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
```

### Choosing weight_decay

1. **Start with 1e-4** as a default
2. **Increase if overfitting** persists
3. **Decrease if underfitting** (training loss too high)
4. **Use AdamW** instead of Adam for better results

### When to Use L2 Regularization

- Almost always! It's a safe default
- Especially with small datasets
- When model has many parameters
- Combine with dropout for best results